# Day 17: Trick Shot
https://adventofcode.com/2021/day/17

### Part 1
What is the highest y position it reaches on this trajectory?

In [1]:
from dataclasses import dataclass, field, InitVar
from typing import Optional, Tuple

%load_ext blackcellmagic

In [2]:
@dataclass(frozen=True, order=True, eq=True)
class Location:
    """Location object"""

    x: int = field(compare=False)
    y: int = field(compare=True)


@dataclass
class Target:
    """Targt object."""

    minx: InitVar[int]
    maxx: InitVar[int]
    miny: InitVar[int]
    maxy: InitVar[int]
    upper_left: Location
    bottom_right: Location

    def __init__(self, minx: int, maxx: int, miny: int, maxy: int):
        self.upper_left = Location(min(minx, maxx), max(miny, maxy))
        self.bottom_right = Location(max(minx, maxx), min(miny, maxy))

    def in_target_area(self, l: Location) -> bool:
        """
        Determins if the location, l, is within the boundaries of the
        target. Returns True if yes, False if not.
        """
        if (
            self.upper_left.x <= l.x <= self.bottom_right.x
            and self.bottom_right.y <= l.y <= self.upper_left.y
        ):
            return True
        else:
            return False


@dataclass
class Probe:
    """
    Probe object.
    Simulates a probe fire, given a starting point of 0,0 and a
    starting x and y velocity.
    """

    velocity_x: int = field(default=0, repr=False, compare=False)
    velocity_y: int = field(default=0, repr=False, compare=False)
    steps: int = field(default=0, repr=False, compare=False)
    locations: Optional[list[Location]] = field(default_factory=list)

    def __post_init__(self):
        self.locations = []
        self.locations.append(Location(0, 0))
        self._step()

    def _step(self, drag: int = -1, gravity: int = -1):
        """make step"""
        last_location = self.locations[-1]
        new_location = Location(
            last_location.x + self.velocity_x, last_location.y + self.velocity_y
        )
        self.locations.append(new_location)
        self.steps = self.steps + 1

        self.velocity_x = (
            self.velocity_x + drag if self.velocity_x > 0 else self.velocity_x
        )
        self.velocity_y = self.velocity_y + gravity

    def fire(self, t: Target) -> bool:
        """
        Fires probe. Return True if eventually hits the target, or False
        if it passes the targetby
        """
        if self.hit_target(t):
            return True
            
        while (
            self.locations[-1].x <= t.bottom_right.x
            and self.locations[-1].y >= t.bottom_right.y
        ):
            self._step()
            if self.hit_target(t):
                return True
        return False

    def hit_target(self, t: Target) -> bool:
        """
        Is the last location within the target, t? True if yes, False 
        if not.
        """
        return t.in_target_area(self.locations[-1])

    def max_height(self):
        """
        Returns the location with the max height from the locations on
        the trajectory.
        """
        return max([height for height in self.locations])

In [3]:
# Unit testing the code we have with the samples provided in
# the text.

test_target = Target(20, 30, -10, -5)

probe = Probe(7, 2)
assert probe.fire(test_target) == True  # hits
assert len(probe.locations) == 8  # reaches in 8 steps (incl. orig 0,0)

probe = Probe(17, -4)
assert probe.fire(test_target) == False  # misses

probe = Probe(6, 9)
assert probe.fire(test_target) == True  # hits
assert probe.max_height().y == 45  # reaches height of 45

In [4]:
def sweep(t: Target) -> Tuple[int, int]:
    """
    Brute force sweep of the space, return x and y with the
    max height.
    """

    max_x = None
    max_y = None
    max_height: Location = Location(0, 0)

    for velocity_x in range(1, t.bottom_right.x + 1, 1):
        for velocity_y in range(t.bottom_right.y, abs(t.bottom_right.y), 1):
            probe = Probe(velocity_x, velocity_y)
            hits: bool = probe.fire(t)
            if hits and probe.max_height() > max_height:
                # print((velocity_x, velocity_y), max_height)
                max_x = velocity_x
                max_y = velocity_y
                max_height = probe.max_height()
    return max_x, max_y


x, y = sweep(Target(20, 30, -10, -5))
p = Probe(x, y)
p.fire(Target(20, 30, -10, -5))
print((x, y), p.max_height())

(6, 9) Location(x=21, y=45)


In [5]:
target = Target(48, 70, -189, -148)
x, y = sweep(target)
p = Probe(x,y)
p.fire(target)

# Print the initial velocity as a tuple and the location
# with the highest total height.
print((x, y), p.max_height())

(10, 188) Location(x=55, y=17766)


### Part 2

In [6]:
def sweep_all(t: Target) -> int:
    """
    Brute force sweep of the space, return a count of x and y
    initial velocities that will hit the target.
    """
    count: int = 0
    for velocity_x in range(1, t.bottom_right.x+1, 1):
        for velocity_y in range(t.bottom_right.y-1, abs(t.bottom_right.y)+1, 1):
            if Probe(velocity_x, velocity_y).fire(t):
                count = count + 1
    return count

# unit test our code.
sweep_all(Target(20, 30, -10, -5)) == 112
sweep_all(Target(48, 70, -189, -148))


1733